In [1]:
import pickle
import json 
import requests
import time
import pandas as pd
from collections import defaultdict as dd
import numpy as np

In [2]:
with open('data/final.pickle', 'rb') as f:
    tmp = pickle.load(f)
    df = pd.DataFrame.from_dict(tmp)
    
# df = pd.read_csv('data/final.pickle')

In [3]:
parents = df[df['parent_ID'] == '-1']

In [4]:
comments = df[df['parent_ID'] != '-1']

In [5]:
pdict = parents.reset_index(drop=True).to_dict()
new = dd()
values = dd()
for i in range(len(pdict['tweet_id'])):
    parent = pdict['tweet_id'][i]
    tmp = comments[comments['parent_ID']==parent]
    new[parent] = tmp.tweet_raw_text.values
    values[parent] = pdict['task_1'][i]
#     tmp1.append(tmp.shape[0])
#     break

In [6]:
ones = 0
zeros = 0
for i in values.keys():
    if values[i] == '1':
        ones += 1
    else:
        zeros += 1
ones, zeros

(337, 358)

In [7]:
count = 0
counts = []
for i in new.keys():
    tmp = []
    for j in new[i]:
        if '&gt' not in j or j.strip() != '':
            tmp.append([count, j])
            count += 1
    counts.append(len(tmp))
    new[i] = tmp
#     print(new[i])
#     break
count

26144

In [8]:
with open('data/perspective_scores.pickle','rb') as f:
    pers_scores = pickle.load(f)
    print(pers_scores.keys())

dict_keys(['TOXICITY_WHOLE', 'TOXICITY_RAW', 'SEVERE_TOXICITY_WHOLE', 'SEVERE_TOXICITY_RAW', 'TOXICITY_FAST_WHOLE', 'TOXICITY_FAST_RAW', 'IDENTITY_ATTACK_WHOLE', 'IDENTITY_ATTACK_RAW', 'INSULT_WHOLE', 'INSULT_RAW', 'PROFANITY_WHOLE', 'PROFANITY_RAW', 'THREAT_WHOLE', 'THREAT_RAW', 'SEXUALLY_EXPLICIT_WHOLE', 'SEXUALLY_EXPLICIT_RAW', 'OBSCENE_WHOLE', 'OBSCENE_RAW', 'RAW_SPAN', 'WHOLE_SPAN'])


In [9]:
def dflt():
    return -1
deleted = []
for i in pers_scores.keys():
    if len(pers_scores[i]) == 0:
        deleted.append(i)
    else:
        pers_scores[i] = dd(dflt, pers_scores[i])
for i in deleted:
    pers_scores.pop(i)

In [10]:
print(pers_scores.keys())

dict_keys(['TOXICITY_WHOLE', 'SEVERE_TOXICITY_WHOLE', 'TOXICITY_FAST_WHOLE', 'IDENTITY_ATTACK_WHOLE', 'INSULT_WHOLE', 'PROFANITY_WHOLE', 'THREAT_WHOLE', 'SEXUALLY_EXPLICIT_WHOLE', 'OBSCENE_WHOLE'])


In [79]:
final_scores = {
    'ids' : [],
    'targets' : [],
    'scores' : []
}
for i in new.keys():
    comments_score = []
    cmt_count = 0
    for j in new[i]:
        if pers_scores['TOXICITY_WHOLE'][j[0]]!= -1:
            cmt_count += 1
            tmp1 = []
            for k in pers_scores.keys():
                tmp1.append(pers_scores[k][j[0]])
            comments_score.append(tmp1)
    comments_score = np.array(comments_score)
    final_scores['ids'].append(i)
    final_scores['targets'].append(values[i])
    final_scores['scores'].append(np.mean(comments_score, axis=0))
#     break

/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [80]:
import pandas as pd
import xgboost
import re
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [81]:
def train_model(classifier, feature_vector_train, label,  feature_vector_valid, valid_y,test_data , test_label ,is_neural_net=False, xgb = False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    #print("In Validation Data",metrics.accuracy_score(predictions, valid_y))
    #applying in test data
    predictions_test = classifier.predict(test_data)
    
    if is_neural_net:
        predictions_test = predictions_test.argmax(axis=-1)
    print("f1 score: ",f1_score(test_label,predictions_test))
        
    return classifier, f1_score(test_label,predictions_test), metrics.accuracy_score(test_label,predictions_test)
    

In [82]:
for i in range(len(final_scores['scores'])):
    if final_scores['targets'][i] == '1':
        final_scores['targets'][i] = 1
    else:
        final_scores['targets'][i] = 0

In [84]:
for i in range(len(final_scores['scores'])):
    if final_scores['scores'][i].shape == ():
        del final_scores['ids'][i]
        del final_scores['targets'][i]
        del final_scores['scores'][i]

In [85]:
# split the dataset into training and validation datasets 
from sklearn.model_selection import train_test_split
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['tweet'], train['label'])
train_x, valid_x, train_y, valid_y = train_test_split(final_scores['scores'], final_scores['targets'], test_size = 0.2, random_state = 42)

In [90]:
accuracy = train_model(linear_model.LogisticRegression(), train_x, train_y, valid_x, valid_y, valid_x, valid_y)
print ("NB, WordLevel TF-IDF: ", accuracy)

f1 score:  0.7703703703703704
NB, WordLevel TF-IDF:  (LogisticRegression(), 0.7703703703703704, 0.7769784172661871)


In [91]:
accuracy = train_model(ensemble.RandomForestClassifier(), train_x, train_y, valid_x, valid_y, valid_x, valid_y)
print ("NB, WordLevel TF-IDF: ", accuracy)

f1 score:  0.881118881118881
NB, WordLevel TF-IDF:  (RandomForestClassifier(), 0.881118881118881, 0.8776978417266187)


In [92]:
accuracy = train_model(svm.SVC(), train_x, train_y, valid_x, valid_y, valid_x, valid_y)
print ("NB, WordLevel TF-IDF: ", accuracy)

f1 score:  0.9090909090909092
NB, WordLevel TF-IDF:  (SVC(), 0.9090909090909092, 0.9064748201438849)


In [59]:
np.stack(train_x, axis = 0).shape

ValueError: all input arrays must have the same shape

In [60]:
for i in train_x:
    print(i.shape)
#     break

(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
